количество кадров в файле CNTRL.mp4 длительностью 5 минут с помощью библиотеки OpenCV

In [5]:
import cv2
import os
from ultralytics import YOLO

In [1]:
# Загрузка видеофайла
video = cv2.VideoCapture("CNTRL.mp4")

# Получение общего количества кадров
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Вывод количества кадров
print("Количество кадров в видеофайле:", total_frames)

# Освобождение ресурсов
video.release()

Количество кадров в видеофайле: 7509


В идеале надо аннотировать каждый кадр, что очень трудозатратно. Поэтому для совсем пробного варианта выберем каждый 75-й кадр (а потом уже за каждую секунду - каждый 25 кадр). Соответственно - получаем короткое видео.

In [2]:
# Имя и путь к исходному видеофайлу
video_path = "CNTRL.mp4"

# Шаг для выборки кадров
step = 75

# Открытие видеофайла
video = cv2.VideoCapture(video_path)

# Общее количество кадров в видеофайле
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Создание объекта VideoWriter для сохранения выбранных кадров в новом видеофайле
output_video = cv2.VideoWriter("sampled_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (int(video.get(3)),int(video.get(4))))

# Чтение и сохранение кадров с шагом 75
for i in range(0, total_frames, step):
    video.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = video.read()
    if ret:
        output_video.write(frame)

# Освобождение ресурсов
video.release()
output_video.release()
cv2.destroyAllWindows()

сохраняем каждый кадр из нового видео "sampled_video.mp4" в отдельный файл JPG. Каждое изображение сохраняется в папке "frames" под именем вида "frame_0.jpg", "frame_1.jpg", и так далее. Этот набор изображений можно будет использовать для аннотирования.

In [4]:
# Открытие видеофайла
video = cv2.VideoCapture("sampled_video.mp4")

# Создание папки для сохранения изображений
if not os.path.exists("frames"):
    os.makedirs("frames")

# Счетчик для именования файлов
frame_counter = 0

while True:
    # Чтение кадра из видео
    ret, frame = video.read()
    if not ret:
        break

    # Сохранение кадра в файл формата JPG
    cv2.imwrite(os.path.join("frames", f"frame_{frame_counter}.jpg"), frame)

    # Увеличение счетчика кадров
    frame_counter += 1

# Освобождение ресурсов
video.release()
cv2.destroyAllWindows()

Нужно сделать аннотирование (отметить СТРОПЫ на кадрах) ранее сохраненных кадров в Roboflow для получения базы data.yaml и дообучить YOLOv8 на этой базе, чтобы получить веса веса дообученной модели.

In [ ]:
## ЗДЕСЬ ДОЛЖЕН БЫТЬ код обучения 

In [ ]:
""" 
# Загрузка YOLOv8 модели
model = YOLO("yolov8m.pt")

# Загрузка видеофайла
video_path = "sampled_video.mp4"

# Обработка видео с помощью модели YOLOv8
results = model(video_path)

# Отображение видео с результатами
results.show()
 """

* ниже код по детекции стандартной модели YOLOv8 (может опрелить 80 видов объектов), которая определяет на каждом кадре видео объекты и рисует рамки вокруг них (с координатами и вероятностью). 
* Далее получаем новое видео уже с этой детекцией. 
* Подобно этому после получения обученной модели на аннотированной базе по стропам мы запустим эту же обработку и в итоге получим детекцию строп в рамках с координатами (координаты будем сохранять для дальнейшей обработки). 
* В следующем блоке обработаем базу координат, чтобы получить нужные углы.

In [32]:
# Загрузка YOLOv8 модели
model = YOLO("yolov8m.pt")

# Загрузка видеофайла
video_path = "sampled_video.mp4"
cap = cv2.VideoCapture(video_path)

# Получение параметров видео
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Создание объекта для записи видео
output_video = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Обработка каждого кадра видео
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Обнаружение объектов на кадре
    results = model.predict(frame)

    # Отображение результатов на кадре
    for box in results[0].boxes:
        class_id = model.names[box.cls[0].item()]
        cords = [round(x) for x in box.xyxy[0].tolist()]
        conf = round(box.conf[0].item(), 2)
        print("Object type:", class_id)
        print("Coordinates:", cords)
        print("Probability:", conf)
        print("---")

        # Рисование ограничивающей рамки на кадре
        cv2.rectangle(frame, (cords[0], cords[1]), (cords[2], cords[3]), (0, 255, 0), 2)
        cv2.putText(frame, f"{class_id} {conf}", (cords[0], cords[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Запись кадра с результатами в выходное видео
    output_video.write(frame)

# Освобождение ресурсов
cap.release()
output_video.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 2 boats, 1504.6ms
Speed: 112.6ms preprocess, 1504.6ms inference, 27.6ms postprocess per image at shape (1, 3, 384, 640)
Object type: boat
Coordinates: [1, 125, 1153, 708]
Probability: 0.93
---
Object type: boat
Coordinates: [1249, 85, 1279, 145]
Probability: 0.63
---
Object type: person
Coordinates: [93, 72, 122, 133]
Probability: 0.5
---

0: 384x640 1 person, 1 truck, 1 boat, 1516.3ms
Speed: 15.0ms preprocess, 1516.3ms inference, 31.9ms postprocess per image at shape (1, 3, 384, 640)
Object type: person
Coordinates: [1248, 89, 1279, 135]
Probability: 0.34
---
Object type: boat
Coordinates: [3, 124, 1146, 702]
Probability: 0.33
---
Object type: truck
Coordinates: [2, 126, 1148, 695]
Probability: 0.27
---

0: 384x640 1 boat, 1530.0ms
Speed: 18.0ms preprocess, 1530.0ms inference, 30.6ms postprocess per image at shape (1, 3, 384, 640)
Object type: boat
Coordinates: [1, 92, 1204, 696]
Probability: 0.93
---

0: 384x640 1 boat, 974.2ms
Speed: 5.9ms preprocess, 974.2ms i

В этом блоке обработаем базу координат, чтобы получить нужные углы.

In [ ]:
## ЗДЕСЬ ДОЛЖЕН БЫТЬ код расчета углов